# *data load*

In [1]:
import json
import re
from pprint import pprint
import config
import swifter
from pshmodule.utils import filemanager as fm
from pshmodule.processing import processing as p

In [2]:
df = fm.load(config.data_path)

extension : .json
Loaded 23936 records from /home/jovyan/shpark-datashare/news_category/data/git_category.json


# *data processing*

In [3]:
df.head()

,content,label
0,더불어민주당 부동산특별위원회는 주택 공급대책을 추가로 마련하기 위한 논의를 한창 벌...,[28]
1,현대인들의 고민거리 중에 적잖은 부분을 차지하는 것이 탈모다 국민건강보험공단에 따르...,"[18, 27]"
2,문화체육관광부는 한국관광공사와 함께 광역기초자치단체를 대상으로 26일부터 7월 30...,"[6, 16]"
3,토요타의 중형 세단 캠리가 상품성을 개선한 2022년형 모델로 돌아왔다 스포티한 디...,[24]
4,만 6074세 고령층의 코로나19 백신 접종 사전예약률이 꾸준히 올라 50를 눈앞에...,"[19, 25]"


In [4]:
df['content'] = df.content.swifter.apply(p.news_preprocessing)

Pandas Apply:   0%|          | 0/23936 [00:00<?, ?it/s]

In [5]:
df.head()

,content,label
0,더불어민주당 부동산특별위원회는 주택 공급대책을 추가로 마련하기 위한 논의를 한창 벌...,[28]
1,현대인들의 고민거리 중에 적잖은 부분을 차지하는 것이 탈모다 국민건강보험공단에 따르...,"[18, 27]"
2,문화체육관광부는 한국관광공사와 함께 광역기초자치단체를 대상으로 26일부터 7월 30...,"[6, 16]"
3,토요타의 중형 세단 캠리가 상품성을 개선한 2022년형 모델로 돌아왔다 스포티한 디...,[24]
4,만 6074세 고령층의 코로나19 백신 접종 사전예약률이 꾸준히 올라 50를 눈앞에...,"[19, 25]"


# *tokenizer*

In [6]:
def gen():
    for row in df.content:
        yield p.news_preprocessing(row)

In [7]:
vocab_size = 24000

In [8]:
user_defined_symbols = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "[BOS]", "[EOS]", "[TSEP]", "[NAME]", "[URL]"]
user_defined_symbols += [f"[UNK{i}]" for i in range(10)]
unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(100)]
user_defined_symbols += unused_list

pprint(user_defined_symbols)

['[PAD]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '[BOS]',
 '[EOS]',
 '[TSEP]',
 '[NAME]',
 '[URL]',
 '[UNK0]',
 '[UNK1]',
 '[UNK2]',
 '[UNK3]',
 '[UNK4]',
 '[UNK5]',
 '[UNK6]',
 '[UNK7]',
 '[UNK8]',
 '[UNK9]',
 '[UNUSED0]',
 '[UNUSED1]',
 '[UNUSED2]',
 '[UNUSED3]',
 '[UNUSED4]',
 '[UNUSED5]',
 '[UNUSED6]',
 '[UNUSED7]',
 '[UNUSED8]',
 '[UNUSED9]',
 '[UNUSED10]',
 '[UNUSED11]',
 '[UNUSED12]',
 '[UNUSED13]',
 '[UNUSED14]',
 '[UNUSED15]',
 '[UNUSED16]',
 '[UNUSED17]',
 '[UNUSED18]',
 '[UNUSED19]',
 '[UNUSED20]',
 '[UNUSED21]',
 '[UNUSED22]',
 '[UNUSED23]',
 '[UNUSED24]',
 '[UNUSED25]',
 '[UNUSED26]',
 '[UNUSED27]',
 '[UNUSED28]',
 '[UNUSED29]',
 '[UNUSED30]',
 '[UNUSED31]',
 '[UNUSED32]',
 '[UNUSED33]',
 '[UNUSED34]',
 '[UNUSED35]',
 '[UNUSED36]',
 '[UNUSED37]',
 '[UNUSED38]',
 '[UNUSED39]',
 '[UNUSED40]',
 '[UNUSED41]',
 '[UNUSED42]',
 '[UNUSED43]',
 '[UNUSED44]',
 '[UNUSED45]',
 '[UNUSED46]',
 '[UNUSED47]',
 '[UNUSED48]',
 '[UNUSED49]',
 '[UNUSED50]',
 '[UNUSED51]',
 '[UNUSED52]',
 '

# *train*

In [9]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [10]:
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [11]:
from tokenizers.pre_tokenizers import Whitespace

bert_tokenizer.pre_tokenizer = Whitespace()

In [12]:
from tokenizers.processors import TemplateProcessing

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)],
)

In [13]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=vocab_size, 
    special_tokens=user_defined_symbols,
)
bert_tokenizer.train_from_iterator(gen(), trainer)

### test

In [14]:
output = bert_tokenizer.encode("테스트 용 문장")
print(output.ids)

bert_tokenizer.decode(output.ids)

[2, 4153, 1476, 9218, 261, 3]


'테스트 용 문자 ##ᆼ'

In [15]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'테스트 용 문장'

# 

# convert transformers tokenizer and save

In [16]:
from transformers import ElectraTokenizerFast


fast_tokenizer = ElectraTokenizerFast(tokenizer_object=bert_tokenizer)

In [17]:
fast_tokenizer.pad_token = "[PAD]"
fast_tokenizer.unk_token = "[UNK]"
fast_tokenizer.cls_token = "[CLS]"
fast_tokenizer.sep_token = "[SEP]"
fast_tokenizer.mask_token = "[MASK]"
fast_tokenizer.bos_token = "[BOS]"
fast_tokenizer.eos_token = "[EOS]"

special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
fast_tokenizer.add_special_tokens(special_tokens_dict)

0

In [18]:
fast_tokenizer.decode(fast_tokenizer.encode("테스트용"))

'[CLS] 테스트용 [SEP]'

In [19]:
fast_tokenizer.save_pretrained(config.tokenizer_path)

('/home/jovyan/shpark-datashare/news_category/result/tokenizer_wordpiece/tokenizer_config.json',
 '/home/jovyan/shpark-datashare/news_category/result/tokenizer_wordpiece/special_tokens_map.json',
 '/home/jovyan/shpark-datashare/news_category/result/tokenizer_wordpiece/vocab.txt',
 '/home/jovyan/shpark-datashare/news_category/result/tokenizer_wordpiece/added_tokens.json',
 '/home/jovyan/shpark-datashare/news_category/result/tokenizer_wordpiece/tokenizer.json')